<a href="https://colab.research.google.com/github/PeterTKovacs/mosquito_recoginition/blob/master/APRO_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from PIL import Image
from pathlib import Path
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
import os
from os.path import isfile, join
import random

import numpy as np
import torch
import torchvision

from torchvision import transforms
from tqdm import tqdm

In [10]:
class BaseLine:
    def __init__(self,
        input_images_path_ = 'drive/My Drive/AML2020/images',
        csv_ = 'drive/My Drive/AML2020/train.csv',
        input_path_ = 'drive/My Drive/AML2020',
        train_path_ = 'drive/My Drive/AML2020/train4',
        test_path_ = 'drive/My Drive/AML2020/test',
        valid_path_ = 'drive/My Drive/AML2020/validation',
                no_=4):
        
        import torch
        import torchvision
        
        self.input_images_path = input_images_path_
        self.csv = csv_
        
        self.input_path = input_path_
        self.train_path = train_path_
        self.test_path = test_path_
        self.valid_path = valid_path_
        self.no=no_


    
    def folder_generator(self):
        import pandas as pd
        import logging as log
        import os
        
        csv_labels = pd.read_csv(self.csv)
        
        #images_paths = list(csv_labels['file'])
        images_paths = csv_labels['file']
        images_paths = [path.replace('images/', '') for path in images_paths]
        
        #images_labels = list(csv_labels['is_tiger'])
        images_labels = csv_labels['is_tiger']
        counter = 1
        
        tiger_path_train = os.path.join(self.train_path, 'tiger')
        tiger1_path_train = os.path.join(self.train_path, 'tiger1')
        tiger2_path_train = os.path.join(self.train_path, 'tiger2')
        tiger3_path_train = os.path.join(self.train_path, 'tiger3')
        tiger4_path_train = os.path.join(self.train_path, 'tiger4')
        not_tiger_path_train = os.path.join(self.train_path, 'not_tiger')

        
        tiger_path_val = os.path.join(self.valid_path, 'tiger')
        not_tiger_path_val = os.path.join(self.valid_path, 'not_tiger')
        
        test_unknown_path = os.path.join(self.test_path, 'unknown')
        
        os.mkdir(self.train_path)
        os.mkdir(tiger_path_train)
        os.mkdir(not_tiger_path_train)

        os.mkdir(self.valid_path)
        os.mkdir(tiger_path_val)
        os.mkdir(not_tiger_path_val)
        
        os.mkdir(self.test_path)
        os.mkdir(test_unknown_path)
        
        #Handling Train Images
        print('###Handling Train Images')
        for image_name, image_label in zip(images_paths, images_labels):
            print('Copying image: {}, with label: {}'.format(image_name, image_label))    
            if image_name.split('_')[0] == 'train' and counter % 5 != 0:
                if image_label == 1:
                    source_dir = os.path.join(self.input_images_path, image_name) 
                    dest_dir = os.path.join(tiger_path_train, image_name)
                    shutil.copy(source_dir, dest_dir)
                    counter += 1
                    print('Copied image: {} to {}'.format(image_name, tiger_path_train))
                else:
                    source_dir = os.path.join(self.input_images_path, image_name) 
                    dest_dir = os.path.join(not_tiger_path_train, image_name)
                    shutil.copy(source_dir, dest_dir)
                    counter += 1
                    print('Copied image: {} to {}'.format(image_name, not_tiger_path_train))
            else:
                if image_label == 1:
                    source_dir = os.path.join(self.input_images_path, image_name) 
                    dest_dir = os.path.join(tiger_path_val, image_name)
                    shutil.copy(source_dir, dest_dir)
                    counter += 1
                    print('Copied image: {} to {}'.format(image_name, tiger_path_val))
                else:
                    source_dir = os.path.join(self.input_images_path, image_name) 
                    dest_dir = os.path.join(not_tiger_path_val, image_name)
                    shutil.copy(source_dir, dest_dir)
                    counter += 1
                    print('Copied image: {} to {}'.format(image_name, not_tiger_path_val))
            
    
    def data_loader(self):
        import torch
        import torchvision
        from torchvision import transforms
        #ImageFolder
        #Augementation for train and valid images
        train_transforms = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        val_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(), 
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        train_dataset = torchvision.datasets.ImageFolder(self.train_path, train_transforms)
        #train_dataset = torchvision.datasets.ImageFolder(train_data, train_transforms)
        validation_dataset = torchvision.datasets.ImageFolder(self.valid_path, val_transforms)
        
        #DataLoader
        batch_size = 16
        train_dataloader = torch.utils.data.DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size
        )

        validation_dataloader = torch.utils.data.DataLoader(
            validation_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size
        )
        print(len(train_dataset), len(validation_dataset))
        print(len(train_dataloader), len(validation_dataloader))
        return train_dataloader, validation_dataloader
    
    def show_images(self, train_dataloader):
        import numpy as np
        import matplotlib.pyplot as plt
        X_batch, y_batch = next(iter(train_dataloader))
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([[0.229, 0.224, 0.225]])
        for index in range(5):
            plt.title(y_batch[index])
            plt.imshow(X_batch[index].permute(1,2,0).numpy() * std + mean, )
            plt.show()
    
    def model(self):
        from torchvision import models
        import torch
        model = models.resnet152(pretrained=True)
        
        
        #Disable grad for all conv layers
        for param in model.parameters():
            param.requires_grad = False 

        model.fc = torch.nn.Linear(model.fc.in_features, 2)
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        model.to(device)
        loss = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(),amsgrad=True, lr=3.0e-4)
       
        
        #Declay LR by a factor of 0.1 every 5th epoch
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.4)
        return model, loss, optimizer, scheduler, device
    
    def train_model(self, model, loss, optimizer, scheduler, train, validation, device, num_epochs):
        import torch
        from tqdm import tqdm

        best_accuracy=-1
        best_path='best'+str(self.no)+'.pth'

        for epoch in range(num_epochs):
            print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    dataloader = train
                    scheduler.step()
                    
                    model.train()  # Set model to training mode
                else:
                    dataloader = validation
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.
                running_acc = 0.
                running_acc_tiger = 0.
                running_acc_not_tiger = 0.

                # Iterate over data.
                for inputs, labels in tqdm(dataloader):
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    optimizer.zero_grad()

                    # forward and backward
                    with torch.set_grad_enabled(phase == 'train'):
                        preds = model(inputs)
                        loss_value = loss(preds, labels)
                        preds_class = preds.argmax(dim=1)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss_value.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss_value.item()
                    running_acc += (preds_class == labels.data).float().mean()
                    running_acc_tiger += (preds_class == 1).float().mean()
                    running_acc_not_tiger += (preds_class == 0).float().mean()

                epoch_loss = running_loss / len(dataloader)
                epoch_acc = running_acc / len(dataloader)
                epoch_acc_tiger = running_acc_tiger / len(dataloader)
                epoch_acc_not_tiger = running_acc_not_tiger / len(dataloader)



                print('{} Loss: {:.4f} Acc: {:.4f} Tiger Acc: {:.4f} Not Tiger Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc, epoch_acc_tiger, epoch_acc_not_tiger), flush=True)

                if epoch_acc>best_accuracy and phase=='val' :
                    
                    self.create_checkpoint(model,optimizer,best_path,epoch,loss_value)
                    best_accuracy=epoch_acc
        
        
        self.load_from_checkpoint(model,optimizer,best_path)

        return model

    def create_checkpoint(self,model,optimizer,path,epoch,loss):
        
        # basically copied from the tutorial, 
        # https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html

        torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': loss,
              }, path)
        
    def load_from_checkpoint(self,model,optimizer,path):
        
        # basically copied from the tutorial, 
        # https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html
        
        checkpoint = torch.load(path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        print('reloading best model of epoch %d, loss %f'%(checkpoint['epoch'],checkpoint['loss']))

    
    def run(self,num_ep=10):
        #Processing data
        #Job = BaseLine()
        #Job.folder_generator()
        #train_dataloader, validation_dataloader = Job.data_loader('drive/My Drive/AML2020/train0')
        #train_dataloader1, validation_dataloader1 = Job.data_loader('drive/My Drive/AML2020/train1')
        #train_dataloader2, validation_dataloader2 = Job.data_loader('drive/My Drive/AML2020/train2')
        #train_dataloader3, validation_dataloader3 = Job.data_loader('drive/My Drive/AML2020/train3')
        #train_dataloader4, validation_dataloader4 = Job.data_loader('drive/My Drive/AML2020/train4')
        #Job.show_images(train_dataloader)
        
        #Define model
        #model, loss, optimizer, scheduler, device = Job.model()
        #print(model, loss, optimizer, scheduler)
        #Train model 
        #trained_model = Job.train_model(model, loss, optimizer, scheduler, train_dataloader, validation_dataloader, device, num_epochs=num_ep)
        #trained_model1 = Job.train_model(model, loss, optimizer, scheduler, train_dataloader1, validation_dataloader1, device, num_epochs=num_ep)
        #trained_model2 = Job.train_model(model, loss, optimizer, scheduler, train_dataloader2, validation_dataloader2, device, num_epochs=num_ep)
        #trained_model3 = Job.train_model(model, loss, optimizer, scheduler, train_dataloader3, validation_dataloader3, device, num_epochs=num_ep)
        #trained_model4 = Job.train_model(model, loss, optimizer, scheduler, train_dataloader4, validation_dataloader4, device, num_epochs=num_ep)
        #return trained_model, trained_model1, trained_model2, trained_model3, trained_model4
        #return trained_model



        train_dataloader, validation_dataloader = self.data_loader()
        
        #Define model
        
        model, loss, optimizer, scheduler, device = self.model()

        #Train model 
        
        trained_model = self.train_model(model, loss, optimizer, scheduler, train_dataloader, validation_dataloader,
                                         device, num_epochs=num_ep)
        return trained_model

In [11]:
model = BaseLine().run()

1132 855
71 54
Epoch 0/9:


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
100%|██████████| 71/71 [00:40<00:00,  1.77it/s]

train Loss: 0.6313 Acc: 0.6291 Tiger Acc: 0.5713 Not Tiger Acc: 0.4287



100%|██████████| 54/54 [00:08<00:00,  6.01it/s]

val Loss: 0.8789 Acc: 0.3218 Tiger Acc: 0.1682 Not Tiger Acc: 0.8318


...

100%|██████████| 71/71 [00:11<00:00,  6.05it/s]

train Loss: 0.4037 Acc: 0.8245 Tiger Acc: 0.5114 Not Tiger Acc: 0.4886



100%|██████████| 54/54 [00:09<00:00,  5.94it/s]

val Loss: 0.3774 Acc: 0.8370 Tiger Acc: 0.7305 Not Tiger Acc: 0.2695


reloading best model of epoch 8, loss 0.180708


In [12]:
def model_(n,basedir='drive/My Drive/AML2020/'):
    
    
    
    model=BaseLine(
    train_path_=os.path.join(basedir,'train'+str(n)),
    valid_path_=os.path.join(basedir,'validation'),
    test_path_=os.path.join(basedir,'test'),
    no_=n)
    
    return model

In [13]:
models={}
for i in range(5):
    m=model_(i)
    trained_m=m.run(num_ep=0)
    models[i]=trained_m

1136 855
71 54
reloading best model of epoch 1, loss 0.328908
1132 855
71 54
reloading best model of epoch 8, loss 0.333934
1132 855
71 54
reloading best model of epoch 2, loss 0.304487
1132 855
71 54
reloading best model of epoch 2, loss 0.466655
1132 855
71 54
reloading best model of epoch 8, loss 0.180708


In [14]:
import pandas as pd
import pickle

def softmax(tensor):
    
    z=np.sum(np.exp(np.asarray(tensor)))
    
    return np.exp(np.asarray(tensor))/z


def confidency(preds):
    
    confidencies=[]
    
    for pic in range(preds.shape[0]):
        
        confidencies.append(softmax(preds[pic,:])[1]) # guess that this is the tiger probability
        
    return confidencies

In [92]:
test_tiger_names_all=os.listdir('drive/My Drive/AML2020/validation/tiger')
test_other_names=os.listdir('drive/My Drive/AML2020/validation/not_tiger')
test_tiger_names=random.sample(os.listdir('drive/My Drive/AML2020/validation/tiger'),len(test_other_names))

batch_size=10
counter=0
to_batch=[]
results={}

val_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(), 
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

for id_,model in models.items():
    results[id_]={}
    model.eval()

for pic in test_other_names:
    to_batch.append((pic,val_transforms(Image.open('drive/My Drive/AML2020/validation/not_tiger/'+pic))))
    counter+=1
    
    if counter==batch_size:
        
        batch=torch.cat([(item[1].reshape(-1,3,224,224)).cuda() for item in to_batch ])
        
        for id_,model in models.items():
            
            conf=confidency(model(batch).cpu().detach().numpy())
            
            for idx,item in enumerate(to_batch):
            
                results[id_][item[0]]=conf[idx]
                
        counter=0
        to_batch=[]
        
for pic in test_tiger_names:
    to_batch.append((pic,val_transforms(Image.open('drive/My Drive/AML2020/validation/tiger/'+pic).convert('RGB'))))
    counter+=1
    
    if counter==batch_size:
        
        batch=torch.cat([(item[1].reshape(-1,3,224,224)).cuda() for item in to_batch ])
        
        for id_,model in models.items():
            
            conf=confidency(model(batch).cpu().detach().numpy())
            
            for idx,item in enumerate(to_batch):
            
                results[id_][item[0]]=conf[idx]
                
        counter=0
        to_batch=[]
    
            
if len(to_batch)>0: #some pics remained unevaluated
    
    batch=torch.cat([(item[1].reshape(-1,3,224,224)).cuda() for item in to_batch ])
        
    for id_,model in models.items():

        conf=confidency(model(batch).cpu().detach().numpy())

        for idx,item in enumerate(to_batch):

            results[id_][item[0]]=conf[idx]

    counter=0
    to_batch=[]

In [16]:
f=open('results_dictionary.txt','wb')


In [ ]:
pickle.dump(results,f)

In [93]:
import copy

In [94]:
rr=copy.deepcopy(results)

In [95]:
for idx,model in models.items():
    for key,value in rr[idx].items():
        rr[idx][key]=list([value,])

In [96]:
pd.DataFrame.from_dict(rr[0])

,train_14.png,train_24.png,train_49.png,train_104.png,train_199.png,train_244.png,train_249.png,train_314.png,train_364.png,train_384.png,train_394.png,train_424.png,train_474.png,train_479.png,train_529.png,train_534.png,train_539.png,train_659.png,train_719.png,train_724.png,train_789.png,train_834.png,train_869.png,train_1004.png,train_1029.png,train_1034.png,train_1079.png,train_1084.png,train_1109.png,train_1124.png,train_1134.png,train_1139.png,train_1149.png,train_1179.png,train_1189.png,train_1229.png,train_1294.png,train_1299.png,train_1309.png,train_1339.png,...,train_3059.png,train_1064.png,train_2304.png,train_1989.png,train_2269.png,train_1834.png,train_1859.png,train_1604.png,train_1579.png,train_2444.png,train_2989.png,train_2794.png,train_3359.png,train_2599.png,train_3014.png,train_319.png,train_4154.png,train_2049.png,train_1184.png,train_1659.png,train_654.png,train_2124.png,train_3504.png,train_3144.png,train_174.png,train_324.png,train_3784.png,train_3184.png,train_2464.png,train_2864.png,train_3304.png,train_4114.png,train_2339.png,train_3379.png,train_4199.png,train_3984.png,train_154.png,train_224.png,train_2789.png,train_1729.png
0,0.209736,0.302952,0.403749,0.794819,0.555901,0.357363,0.421041,0.756719,0.117494,0.484588,0.428709,0.681044,0.732484,0.73926,0.791599,0.261638,0.463211,0.435713,0.427354,0.507778,0.498742,0.481971,0.509859,0.45569,0.304075,0.51793,0.625527,0.568673,0.355695,0.499256,0.518045,0.539013,0.458135,0.245634,0.282906,0.693802,0.766299,0.762627,0.296871,0.778213,...,0.351072,0.945673,0.822857,0.902167,0.858425,0.814976,0.766667,0.753307,0.835444,0.815474,0.844762,0.822735,0.703114,0.44943,0.791587,0.836566,0.881824,0.935365,0.939269,0.885314,0.764981,0.865804,0.628737,0.842267,0.762333,0.90339,0.943659,0.883013,0.929438,0.633835,0.717058,0.678885,0.857498,0.755521,0.893088,0.798566,0.828486,0.807249,0.822476,0.934474


In [97]:
tabular_res=pd.concat([pd.DataFrame.from_dict(rr[i]) for i in range(5)])

In [22]:
tabular_res

,train_14.png,train_24.png,train_49.png,train_104.png,train_199.png,train_244.png,train_249.png,train_314.png,train_364.png,train_384.png,train_394.png,train_424.png,train_474.png,train_479.png,train_529.png,train_534.png,train_539.png,train_659.png,train_719.png,train_724.png,train_789.png,train_834.png,train_869.png,train_1004.png,train_1029.png,train_1034.png,train_1079.png,train_1084.png,train_1109.png,train_1124.png,train_1134.png,train_1139.png,train_1149.png,train_1179.png,train_1189.png,train_1229.png,train_1294.png,train_1299.png,train_1309.png,train_1339.png,...,train_1279.png,train_1284.png,train_1289.png,train_1304.png,train_1314.png,train_1319.png,train_1324.png,train_1329.png,train_1334.png,train_1344.png,train_1349.png,train_1359.png,train_1369.png,train_1374.png,train_1379.png,train_1384.png,train_1389.png,train_1394.png,train_1399.png,train_1404.png,train_1409.png,train_1414.png,train_1419.png,train_1424.png,train_1429.png,train_1434.png,train_1439.png,train_1444.png,train_1449.png,train_1454.png,train_1459.png,train_1469.png,train_1479.png,train_1484.png,train_1489.png,train_1494.png,train_1499.png,train_1504.png,train_1509.png,train_1514.png
0,0.209736,0.302952,0.403749,0.794819,0.555901,0.357363,0.421041,0.756719,0.117494,0.484588,0.428709,0.681044,0.732484,0.739260,0.791599,0.261638,0.463211,0.435713,0.427354,0.507778,0.498742,0.481971,0.509859,0.455690,0.304075,0.517930,0.625527,0.568673,0.355695,0.499256,0.518045,0.539013,0.458135,0.245634,0.282906,0.693802,0.766299,0.762627,0.296871,0.778213,...,0.958302,0.903137,0.915827,0.709422,0.826865,0.761198,0.556625,0.914706,0.804403,0.941148,0.790828,0.975146,0.818374,0.950069,0.698066,0.976259,0.579234,0.924904,0.846861,0.713022,0.909954,0.883279,0.888295,0.850803,0.829015,0.334472,0.582512,0.714281,0.782257,0.709204,0.788649,0.855719,0.867720,0.534554,0.930075,0.605718,0.701639,0.842552,0.860819,0.823419
0,0.140258,0.163531,0.210985,0.759600,0.362302,0.080574,0.220442,0.437065,0.020906,0.220081,0.238849,0.615121,0.687314,0.462181,0.534164,0.133865,0.370064,0.241967,0.146943,0.305404,0.144578,0.314170,0.125502,0.620888,0.100052,0.297949,0.493437,0.404515,0.055006,0.143297,0.139867,0.181729,0.312294,0.106453,0.103080,0.682907,0.718419,0.730659,0.067513,0.880105,...,0.968526,0.950648,0.928355,0.807178,0.851160,0.858070,0.558053,0.912332,0.802267,0.982297,0.839294,0.997551,0.940110,0.962747,0.544839,0.992510,0.574529,0.964537,0.936402,0.710499,0.943100,0.972436,0.842301,0.771888,0.859269,0.288327,0.364556,0.715882,0.573327,0.839148,0.841377,0.882068,0.924490,0.614706,0.944547,0.813964,0.688086,0.886713,0.953959,0.915703
0,0.289888,0.169453,0.446861,0.781015,0.474826,0.208090,0.231537,0.634385,0.080451,0.400536,0.329385,0.706643,0.671806,0.443191,0.601163,0.293394,0.511332,0.328965,0.309848,0.379094,0.265719,0.323338,0.381329,0.432387,0.286150,0.509252,0.474230,0.585852,0.193408,0.332631,0.363611,0.302796,0.404430,0.181492,0.217177,0.739133,0.766585,0.752206,0.206051,0.795320,...,0.942796,0.905074,0.902101,0.772700,0.708904,0.784616,0.528510,0.861282,0.743546,0.964725,0.717268,0.979087,0.863714,0.942825,0.536634,0.969716,0.675530,0.932491,0.856450,0.633190,0.926798,0.830919,0.775776,0.809908,0.856080,0.369874,0.508964,0.774885,0.592175,0.848643,0.834473,0.816150,0.901507,0.660631,0.937495,0.711108,0.780179,0.805622,0.924992,0.842946
0,0.221980,0.161097,0.523462,0.636929,0.343989,0.214602,0.200371,0.557144,0.054974,0.343143,0.367335,0.634559,0.621180,0.529309,0.507047,0.149283,0.346270,0.261988,0.295835,0.348855,0.311465,0.368055,0.340550,0.533217,0.167505,0.431024,0.509012,0.354952,0.152064,0.282278,0.291604,0.244887,0.334682,0.202897,0.167517,0.729442,0.716787,0.556351,0.206270,0.786097,...,0.970260,0.929344,0.907360,0.626607,0.618696,0.781000,0.527184,0.885260,0.813891,0.953549,0.710761,0.986309,0.840814,0.953698,0.602942,0.977843,0.556236,0.896943,0.896644,0.619903,0.899084,0.841505,0.857418,0.831139,0.765478,0.202467,0.454299,0.683115,0.638963,0.813120,0.84782

In [98]:
tabular_res.index=[0,1,2,3,4]
data=tabular_res.transpose()
data.head()

,0,1,2,3,4
train_14.png,0.209736,0.140258,0.289888,0.221980,0.090144
train_24.png,0.302952,0.163531,0.169453,0.161097,0.093939
train_49.png,0.403749,0.210985,0.446861,0.523462,0.145097
train_104.png,0.794819,0.759600,0.781015,0.636929,0.701267
train_199.png,0.555901,0.362302,0.474826,0.343989,0.192143


In [99]:
labels = pd.read_csv('drive/My Drive/AML2020/train.csv')
indeces = []
for pic in list(data.index):
    index = list(labels['file']).index('images/'+pic)
    indeces.append(index)

labels_data = labels.iloc[indeces]
data['labels'] = list(labels_data['is_tiger'])

In [100]:
data.head()

,0,1,2,3,4,labels
train_14.png,0.209736,0.140258,0.289888,0.221980,0.090144,0
train_24.png,0.302952,0.163531,0.169453,0.161097,0.093939,0
train_49.png,0.403749,0.210985,0.446861,0.523462,0.145097,0
train_104.png,0.794819,0.759600,0.781015,0.636929,0.701267,0
train_199.png,0.555901,0.362302,0.474826,0.343989,0.192143,0


In [111]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
X, y = data[[0,1,2,3,4]], data['labels']
clf = LogisticRegression(random_state=0).fit(X,y)
clf2 = svm.SVC().fit(X,y)

In [102]:
weights_tiger=[]
weights_not_tiger=[]
for i in range(len(data)):
  if data['labels'][i] == 1:
    weights_not_tiger.append(0)
    weights_tiger.append(1/(sum(data['labels'])))
  else:
    weights_not_tiger.append(1/(len(data) - sum(data['labels'])))
    weights_tiger.append(0)


Logistic regression accuracy

In [110]:
print('Overall accuracy: {:.4f} Not Tiger accuracy: {:.4f} Tiger accuracy: {:.4f}'.format(clf.score(X,y),clf.score(X,y,sample_weight=weights_not_tiger),clf.score(X,y,sample_weight=weights_tiger)))

Overall accuracy: 0.8346 Not Tiger accuracy: 0.8088 Tiger accuracy: 0.8603


SVM accuracy

In [113]:
print('Overall accuracy: {:.4f} Not Tiger accuracy: {:.4f} Tiger accuracy: {:.4f}'.format(clf2.score(X,y),clf2.score(X,y,sample_weight=weights_not_tiger),clf2.score(X,y,sample_weight=weights_tiger)))

Overall accuracy: 0.8603 Not Tiger accuracy: 0.8897 Tiger accuracy: 0.8309
